# An input file is generated for ASAXS data correction

For the ASAXS data correction a file is nessesary which contains a number of experimental information.\
Identifier (Trigger) \
Image file name \
Monitor, Transmission Diode, Acquisition Time \
Identifier for: Empty, Background, Reference to this measurement

## Libraties

In [1]:
import pandas as pd
import os
import csv
import numpy as np
from scipy import optimize
import scipy.constants as con
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm, LinearSegmentedColormap

## Read the ASCII files

In [2]:
# import methadata
# give the methadada directory
data_dir = 'files\Data_for_ASAXS'

# give the target file tyle:
target_file_type = '.dat'

# The header contains lines with general informations and a long table of all data points with a table header line
# give the number of general header lineshere
general_header_lines = 375


# scan the directory
fileList = list()
for entry in os.scandir(data_dir):
    if entry.path.endswith(target_file_type) and entry.is_file():
        fileList.append(entry.path)
# make an path iterator from the found files
fileIterator = iter(fileList)

# generate the first pandas frame from the first entrie
element = next(fileIterator)
path_to_data = element
dF = pd.read_csv(path_to_data, sep='\t',header=general_header_lines)
dF = dF.drop([0]) # drop the NaN lines

# append all other files
for element in fileIterator:
    path_to_data = element
    print(element)
    new_dF = pd.read_csv(path_to_data, sep='\t',header=general_header_lines)
    new_dF = new_dF.drop([0]) # drop the NaN lines
    dF = dF.append(new_dF)

# suggesting there is a '# ' directly before the first header entry, drop it
dfInitialColumnName = dF.columns[0]
dfNewColumnName = dF.columns.tolist()[0][2:]
dF = dF.rename(columns={dfInitialColumnName: dfNewColumnName}) # get rid of the '# ' in the first column name

# re-index the data frame 0 to the end
dF = dF.reset_index(drop=True)

print('Number of entries:', len(dF))

files\Data_for_ASAXS\fcm_2020kw29tg14_00093.dat
files\Data_for_ASAXS\fcm_2020kw29tg14_00095.dat
files\Data_for_ASAXS\fcm_2020kw29tg14_00097.dat
files\Data_for_ASAXS\fcm_2020kw29tg14_00099.dat
files\Data_for_ASAXS\fcm_2020kw29tg14_00101.dat
files\Data_for_ASAXS\fcm_2020kw29tg14_00103.dat
files\Data_for_ASAXS\fcm_2020kw29tg14_00104.dat
Number of entries: 144


In [9]:
path_to_data

'files\\Data_for_ASAXS\\fcm_2020kw29tg14_00104.dat'

In [94]:
missingInformation = ['Energy']
seperatorBeforeInfo = '\t'
seperatorAfterInfo = '\t = '
digitsAfterIdentifier = 12 # this is the length of the entry after the identifier you are searching for

for element in missingInformation:
    searchString = seperatorBeforeInfo + element + seperatorAfterInfo
    InfoStringLength = len(element) + len(seperatorAfterInfo)
    
    dataFile = open(path_to_data, 'r')
    resultPosition = dataFile.read().find(searchString)
    
    if resultPosition != -1:
        dataFile = open(path_to_data, 'r')
        longText = dataFile.read(resultPosition+InfoStringLength+digitsAfterIdentifier)
        result = longText[resultPosition+InfoStringLength:]
        print(result)
    else:
        print('Missing information is not in the file :(')
    dataFile.close()

 9999.991504


In [96]:
float(result)

9999.991504

Try to define a list of nessesary entries which are taken collected from the tables and header of the files

In [3]:
dF

,PilatusAcqTime,VacSampleX,Sample-Z,Tilt-Z,Sample-Y,Tilt-Y,2Theta,VacSampleY,Keysight1,Keysight2,...,Keysight2:Count,Keysight2:TriggerIntv,Keysight2:StandardDeviation,Keysight3:Count,Keysight3:TriggerIntv,Keysight3:StandardDeviation,Keysight4:Count,Keysight4:TriggerIntv,Keysight4:StandardDeviation,PosCounter
0,600.0,133.00,0.0,0.0,5.0,0.0,-25.0,-1.100000e-05,2.299318e-06,5.430643e-10,...,601.0,1.0,2.695514e-11,601.0,1.0,2.460055e-14,601.0,1.0,3.442476e-10,4
1,300.0,123.00,0.0,0.0,5.0,0.0,-25.0,-4.000000e-06,2.152853e-06,5.432002e-10,...,301.0,1.0,2.427762e-11,301.0,1.0,2.329991e-14,301.0,1.0,3.704751e-10,5
2,10.0,113.00,0.0,0.0,5.0,0.0,-25.0,4.163336e-17,1.337529e-06,5.675809e-10,...,11.0,1.0,1.686306e-11,11.0,1.0,1.217504e-14,11.0,1.0,6.612729e-11,6
3,600.0,88.16,0.0,0.0,5.0,0.0,-25.0,1.000000e-06,1.448823e-06,5.820030e-10,...,601.0,1.0,3.077202e-11,601.0,1.0,2.014422e-14,601.0,1.0,1.552321e-09,7
4,600.0,83.10,0.0,0.0,5.0,0.0,-25.0,4.163336e-17,9.860936e-07,5.657099e-10,...,601.0,1.0,2.458183e-11,601.0,1.0,2.254677e-14,601.0,1.0,4.827976e-10,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139,300.0,23.00,0.0,0.0,5.0,0.0,-25.0,9.999880e-01,4.484193e-07,5.893668e-10,...,300.0,1.0,2.384017e-11,300.0,1.0,2.067708e-14,300.0,1.0,3.907890e-11,17
140,300.0,23.00,0.0,0.0,5.0,0.0,-25.0,-9.149983e+00,4.445866e-07,6.032399e-10,...,301.0,1.0,2.439987e-11,301.0,1.0,2.123514e-14,301.0,1.0,3.485907e-11,18
141,600.0,13.00,0.0,0.0,5.0,0.0,-25.0,7.799996e+00,4.389022e-07,6.102984e-10,...,601.0,1.0,3.592902e-11,601.0,1.0,2.537102e-14,601.0,1.0,3.793525e-11,19
142,600.0,13.00,0.0,0.0,5.0,0.0,-25.0,5.880000e-04,4.517064e-07,5.958377e-10,...,601.0,1.0,3.080868e-11,601.0,1.0,2.876408e-14,601.0,1.0,3.933294e-11,20


In [2]:
# give the methadada directory
data_dir = 'files\Data_for_ASAXS'

# give the target file tyle:
target_file_type = '.dat'

# Concerning the header
# Number of header line
header_lines = 375
# Structure of information in header
seperatorBeforeInfo = '\t'
seperatorAfterInfo = '\t = '
digitsAfterIdentifier = 12 # this is the length of the entry after the identifier you are searching for

# List of experimental divices/information nessesary for the data reduction
deviceList = ['Time','VacSampleX', 'VacSampleY','Energy','Pilatus_Tiff','Pilatus_Trigger','Pilatus_filename','PilatusAcqTime','Keysight1','Keysight2','Keysight3','Keysight4','Keysight1:StandardDeviation','Keysight2:StandardDeviation','Keysight3:StandardDeviation','Keysight4:StandardDeviation','PilatusPos']
#deviceList

In [4]:
# scan the directory
fileList = list()
for entry in os.scandir(data_dir):
    if entry.path.endswith(target_file_type) and entry.is_file():
        fileList.append(entry.path)
# make an path iterator from the found files
fileIterator = iter(fileList)

# generate the first pandas frame from the first entrie
element = next(fileIterator)
path_to_data = element
dF = pd.read_csv(path_to_data, sep='\t',header=header_lines)
dF = dF.drop([0]) # drop the NaN lines

# suggesting there is a '# ' directly before the first header entry, drop it
dfInitialColumnName = dF.columns[0]
dfNewColumnName = dF.columns.tolist()[0][2:]
dF = dF.rename(columns={dfInitialColumnName: dfNewColumnName}) # get rid of the '# ' in the first column name
list_dF_columns = dF.columns.values.tolist()

deviceIterator = iter(deviceList)
device = next(deviceIterator)
if device in list_dF_columns:
    deviceFrame = pd.DataFrame({device:dF[device]})
else:
    print('Error: First device is not in experimental data')

for device in deviceIterator:
    if device in list_dF_columns:
        deviceFrame = deviceFrame.join(dF[device])
    else:
        searchString = seperatorBeforeInfo + device + seperatorAfterInfo
        InfoStringLength = len(device) + len(seperatorAfterInfo)
    
        dataFile = open(path_to_data, 'r')
        resultPosition = dataFile.read().find(searchString)
    
        if resultPosition != -1:
            dataFile = open(path_to_data, 'r')
            longText = dataFile.read(resultPosition+InfoStringLength+digitsAfterIdentifier)
            result = longText[resultPosition+InfoStringLength:]
            deviceFrame[device]=float(result)
            print('Device >', device, '< is taken as constant from the header with a value of',float(result))
        else:
            print('Error: Device >>>',device, '<<< is not in the experimental data and not in the header information :(')
        dataFile.close()

Device > Energy < is taken as constant from the header with a value of 7000.002692


Now for all the remaining files in the directory

In [98]:
for element in fileIterator:
    path_to_data = element
    print(element)
    new_dF = pd.read_csv(path_to_data, sep='\t',header=general_header_lines)
    new_dF = new_dF.drop([0]) # drop the NaN lines
    dF = dF.append(new_dF)

1     7000.002692
2     7000.002692
3     7000.002692
4     7000.002692
5     7000.002692
6     7000.002692
7     7000.002692
8     7000.002692
9     7000.002692
10    7000.002692
11    7000.002692
12    7000.002692
13    7000.002692
14    7000.002692
15    7000.002692
16    7000.002692
17    7000.002692
18    7000.002692
Name: Energy, dtype: float64